In [4]:
import tensorflow as tf
import numpy as np
import csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import wget

In [2]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_token = '<OOV>'
padding_type = 'pre'

In [8]:
tokenizer = Tokenizer()
data = open('irish-lyrics-eof.txt').read()

corpus = data.lower().split('\n')
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

#print(tokenizer.word_index)
print(total_words)
#print(corpus)

2690


In [12]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range (1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

#print(input_sequences)

max_sequence_len = max(len(x) for x in input_sequences )
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen=max_sequence_len,
                                         truncating=trunc_type,
                                         padding=padding_type,
                                         ))

#print(input_sequences)

xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

#print(xs)
#print(labels)

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

#Callbacks = myCallback()

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

history = model.fit(xs, ys, epochs = 100, verbose = 1, )#callbacks=[Callbacks])

Train on 12038 samples
Epoch 1/100
12038/12038 [==============================] - 27s 2ms/sample - loss: 6.6231 - accuracy: 0.0755
Epoch 2/100
12038/12038 [==============================] - 22s 2ms/sample - loss: 5.7507 - accuracy: 0.1150
Epoch 3/100
12038/12038 [==============================] - 21s 2ms/sample - loss: 4.9073 - accuracy: 0.1625
Epoch 4/100
12038/12038 [==============================] - 27s 2ms/sample - loss: 4.0131 - accuracy: 0.2317
Epoch 5/100
12038/12038 [==============================] - 21s 2ms/sample - loss: 3.1842 - accuracy: 0.3335
Epoch 6/100
12038/12038 [==============================] - 21s 2ms/sample - loss: 2.5278 - accuracy: 0.4397
Epoch 7/100
12038/12038 [==============================] - 20s 2ms/sample - loss: 2.0476 - accuracy: 0.5253
Epoch 8/100
12038/12038 [==============================] - 22s 2ms/sample - loss: 1.6725 - accuracy: 0.6057
Epoch 9/100
12038/12038 [==============================] - 22s 2ms/sample - loss: 1.4094 - accuracy: 0.6606
Epoch

In [23]:
model.save("irish_poetry.h5")

In [27]:
import keras
loaded_model = tf.keras.models.load_model('irish_poetry.h5')

In [28]:
def create_poem(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding=padding_type)
        predicted = loaded_model.predict_classes(token_list, verbose=0)
        output_word=''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += ' ' + word
    results = seed_text
    return results

In [29]:
create_poem('Arina is going by the road to Dublin', 40)

'Arina is going by the road to Dublin life has ever a heart of my own gone alas on like our youth too soon deprived in gone and shes gone home gone so mild gone away on the love in my tune before proud saxon lord gone host'